# `dfipy` Quick Start Guide - Large Synthetic Dataset of 92B Records

This notebook will guide you through querying a large synthetic traffic dataset in the General System Platform.

Please refer to our [developers guide](https://developers.generalsystem.com) for the most up-to-date companion documentation.

Additional resources and help are available on the [General System support pages](https://support.generalsystem.com).

### Let's go

### Install Dependencies

If you are using Google Colab, this will set up all required dependencies:

In [ ]:
from copy import deepcopy
from getpass import getpass
from typing import List, Optional, Union

import geopandas as gpd
import pandas as pd
import requests
from IPython.display import Image
from shapely.geometry import Polygon
from tqdm import tqdm

# Google Colab setup
try:
    from google.colab import output

    output.enable_custom_widget_manager()  # allows KeplerGL map to display

    ! pip install dfipy==9.0.1 h3==3.7.6 keplergl==0.3.2

    import dfi.models.filters.geometry as geom
    import h3
    from dfi import Client
    from dfi.models.filters import TimeRange
    from keplergl import KeplerGl

except ModuleNotFoundError:
    import dfi.models.filters.geometry as geom
    import h3
    from dfi import Client
    from dfi.models.filters import TimeRange
    from keplergl import KeplerGl


In [ ]:
boroughs_url = (
    "https://raw.githubusercontent.com/thegeneralsystem/dfipy-examples/main/examples/datasets/london_boroughs.json"
)
congestion_zone_url = "https://raw.githubusercontent.com/thegeneralsystem/dfipy-examples/main/examples/datasets/london_congestion_zone.json"


### Next, enter you API access token

In [ ]:
api_token = getpass("Enter your API access token: ")


### In this tutorial we will be querying a large synthetic data set

This synthetic data set represent traffic moving across London.

| total records	| 92,435,312,835 |
| ------------- | -------------- |
| distinct uuids | 1,578,544 |
| start time | 2022-01-01 00:00:00 |
| end time | 2022-08-26 07:12:00 |

Bounding box of all data:

|      | Longitude  | Latitude |
| ---- | ---------- | -------- |
| Min  | -0.5120832 | 51.2810883 |
| Max  | 0.322123   | 51.6925997 |



#### Hardware
- The dataset runs on a single server hosted on AWS
- The server is storage optimised, with 192GB ram and 2 x 7.5TB NVMe SSD

#### Note: this is a shared instance, and you cannot add or delete data to it.

The following picture shows a heatmap of the London Traffic dataset of 92bn records.

The data distributes along the roads in the city, with Central London having the largest density.

In [ ]:
image_path = (
    "https://raw.githubusercontent.com/thegeneralsystem/dfipy-examples/main/examples/pictures/london_traffic.jpg"
)
Image(image_path)


### Connect to the DFI

In [ ]:
base_url = "https://api.aspen.generalsystem.com"
dataset_id = "gs.prod-3"

dfi = Client(
    api_token=api_token,
    base_url=base_url,
    progress_bar=True,
)


### Define auxiliary methods to aggregate and analyse the datasets.

In what follows, we use H3 spatial indices to aggregate large amounts of data. 

An H3 spatial index allows for efficient spatial referencing, indexing, and analysis of geospatial data at varying resolutions on a global scale. `h3_resolution` refers to the granularity of the aggregation, with higher values providing finer resolution. For more information please see: https://h3geo.org/

In [ ]:
def explode_lat_lon(df: pd.DataFrame) -> pd.DataFrame:
    """Explodes the 'coordinate' column into separate columns for longitude and latitude"""
    df_exploded = df.copy()
    df_exploded[["longitude", "latitude", "altitude"]] = pd.DataFrame(
        df_exploded["coordinate"].tolist(), index=df_exploded.index
    )
    df_exploded.drop(columns=["coordinate", "altitude"], inplace=True)
    return df_exploded


def _aggregate_records(df_input: pd.DataFrame, hex_id: str) -> pd.DataFrame:
    return (
        df_input.groupby(hex_id)
        .agg(
            num_records=("id", "count"),
            num_devices=("id", "nunique"),
            first_ping=("time", "min"),
            last_ping=("time", "max"),
        )
        .reset_index()
    )


def add_heatmap_aggregation(
    df_records: pd.DataFrame,
    h3_resolution: int,
) -> pd.DataFrame:
    return df_records.assign(
        hex_id=lambda df: [
            h3.geo_to_h3(lat, lon, resolution=int(h3_resolution)) for lat, lon in zip(df["latitude"], df["longitude"])
        ]
    ).pipe(_aggregate_records, "hex_id")


def build_heatmap(df_records: pd.DataFrame, h3_resolution: int) -> gpd.GeoDataFrame:
    df_binned_data = add_heatmap_aggregation(df_records=df_records, h3_resolution=h3_resolution)
    hex_geometries = [Polygon(h3.h3_to_geo_boundary(h3_id, geo_json=True)) for h3_id in df_binned_data.hex_id]
    gdf_binned_data = gpd.GeoDataFrame(df_binned_data, geometry=hex_geometries)

    gdf_binned_data_kepler = gdf_binned_data.copy()
    gdf_binned_data_kepler = gdf_binned_data_kepler.drop(columns=["hex_id"])
    gdf_binned_data_kepler.first_ping = gdf_binned_data_kepler.first_ping.astype(str)
    gdf_binned_data_kepler.last_ping = gdf_binned_data_kepler.last_ping.astype(str)
    gdf_binned_data_kepler = gdf_binned_data_kepler.drop(columns=["first_ping", "last_ping"])
    return gdf_binned_data_kepler


In [ ]:
boroughs_request = requests.get(boroughs_url, timeout=30)
boroughs_request.raise_for_status()
boroughs = boroughs_request.json()


In [ ]:
for name, vertices in boroughs.items():
    print(f"{name} - {len(vertices)}")


We can plot the polygons on a map. As an example, we plot the polygon that defines the London Congestion Zone in Central London.

For visualisation we use KeplerGL: a powerful data visualisation software that enables users to explore and analyse large datasets in a visually engaging and intuitive manner. We will be utilising it to visualise the data retrieved to make it easier to share insights gained by DFI. For more information please see: https://kepler.gl/.

An H3 spatial index allows for efficient spatial referencing, indexing, and analysis of geospatial data at varying resolutions on a global scale. `h3_resolution` refers to the granularity of the aggregation, with higher values providing finer resolution. For more information please see: https://h3geo.org/

In [ ]:
def show_map(
    list_polygons: Optional[List[List[List[float]]]] = None,
    list_dfs: Optional[Union[gpd.GeoDataFrame, pd.DataFrame]] = None,
    df_records: Optional[pd.DataFrame] = None,
    map_height: int = 1200,
    config: Optional[dict] = None,
) -> KeplerGl:
    if list_polygons is None:
        list_polygons = []

    dict_polygons = {f"polygon {idx}": poly for idx, poly in enumerate(list_polygons)}

    kepler_data = {}

    if len(dict_polygons) > 0:
        kepler_data.update(
            {
                "polygons": gpd.GeoDataFrame(
                    dict_polygons.keys(),
                    geometry=[Polygon(x) for x in dict_polygons.values()],
                )
            }
        )

    if df_records is not None:
        kepler_data.update({"records": df_records.copy()})

    if list_dfs is not None:
        for idx, df in enumerate(list_dfs):
            kepler_data.update({f"df_{idx}": df.copy()})

    if config is None:
        return KeplerGl(data=deepcopy(kepler_data), height=map_height)
    return KeplerGl(data=deepcopy(kepler_data), height=map_height, config=config)


In [ ]:
congestion_zone_request = requests.get(congestion_zone_url, timeout=30)
congestion_zone_request.raise_for_status()
lon_congestion_zone = congestion_zone_request.json()


In [ ]:
show_map(map_height=400, list_polygons=[lon_congestion_zone])


### Querying the London traffic dataset 

Note: this is a large (92bn) dataset. While it is possible to run queries such as "return all records in this dataset", such queries will be streaming back large amounts of data and will be terminated early to preserve resource in the demo instance. Please include a time interval in your queries to reduce the amount of data streamed back.

Querying spatiotemporal data typically takes hours or days, especially with a point-in-polygon query like this.

Let’s check how many vehicles entered the London Congestion Charging zone during the morning rush hour.

In [ ]:
time_range = TimeRange().from_strings(min_time="2022-01-01T08:00:00+00:00", max_time="2022-01-01T09:30:00+00:00")
congestion_zone_polygon = geom.Polygon().from_raw_coords(coordinates=lon_congestion_zone, geojson=True)

df_congestion_charge_zone = dfi.query.records(
    dataset_id="gs.prod-3", geometry=congestion_zone_polygon, time_range=time_range
)
print(f"Records downloaded: {len(df_congestion_charge_zone):,}")
print(f"Vehicles found: {len(df_congestion_charge_zone.id.unique()):,}")


Let's build a heatmap of the data returned and visualise it on a map

- Cells with darker colours represent areas with less density of records
- Cells with ligther colours represent areas with higher density of records

#### Load map configuration

In [ ]:
url = "https://raw.githubusercontent.com/thegeneralsystem/dfipy-examples/main/examples/kepler_config/syn_traffic.json"
response = requests.get(url, timeout=30)
kepler_config = response.json()


In [ ]:
df_congestion_charge_zone


#### Show the heatmap

In [ ]:
df_latlons = explode_lat_lon(df=df_congestion_charge_zone)
heatmap = build_heatmap(df_records=df_latlons, h3_resolution=10)
map1 = show_map(map_height=400, list_dfs=[heatmap], config=kepler_config)
map1


Let's retrieve the full history of a vehicle and show it on the map

In [ ]:
vehicle = "a37d6189-00ed-4f45-bb6e-aacb1d85090e"
df_history = dfi.query.records(dataset_id="gs.prod-3", uids=[vehicle])
df_latlons = explode_lat_lon(df=df_history)
show_map(
    df_records=df_latlons,
    map_height=400,
)


### Which Boroughs has this vehicle been to?

Next we query each borough to check if the vehicle has visited it

In [ ]:
dfi.conn.progress_bar = False

visited_boroughs = []
for name, vertices in tqdm(boroughs.items()):
    borough_polygon = geom.Polygon().from_raw_coords(coordinates=vertices, geojson=True)
    records = dfi.query.records(dataset_id="gs.prod-3", geometry=borough_polygon, uids=[vehicle])
    count = len(records)
    if count > 0:
        visited_boroughs.append([name])

dfi.conn.progress_bar = True


Finally we print the results

In [ ]:
print(f"{vehicle} has been to {len(visited_boroughs)} / {len(boroughs)} London boroughs")
print("Borough visited:")
for borough in visited_boroughs:
    print(borough)


End of Notebook